In [8]:
# Stage 1: Install & import
!pip install requests pandas -q
import requests
import pandas as pd


In [9]:
# Stage 2: URLs to check
urls_to_check = [
    "http://example.com",
    "http://phishing-test-site.com",
    "https://google.com"
]


In [10]:
# Stage 3: API config (replace with real key to use PhishTank)
API_KEY = ""  # put your PhishTank API key here; leave empty to use local simulated DB
API_URL = "https://checkurl.phishtank.com/checkurl/"


In [11]:
# Stage 4: Local fallback phishing DB (used when no API key)
local_phish_db = {
    "http://phishing-test-site.com",
    "http://test-phish.local/fake-login",
    "http://malicious.example/phish"
}


In [12]:

# Stage 5: Function to check a URL (uses PhishTank if API_KEY set, else local DB)
def check_phish(url):
    if API_KEY:
        params = {"url": url, "format": "json", "app_key": API_KEY}
        try:
            resp = requests.post(API_URL, data=params, timeout=10)
            data = resp.json()
            if "results" in data:
                is_phish = bool(data["results"].get("in_database", False))
                verified = bool(data["results"].get("verified", False))
                return is_phish, verified
            else:
                return False, False
        except:
            return False, False
    else:
        in_db = url in local_phish_db
        verified = in_db
        return in_db, verified


In [15]:
# Stage 6: Run checks and collect results
results = []
for url in urls_to_check:
    phish, verified = check_phish(url)
    results.append({"URL": url, "Phishing": phish, "Verified": verified})
df_results = pd.DataFrame(results)


In [16]:
print(df_results)
df_results.to_csv("phishing_url_results.csv", index=False)

                             URL  Phishing  Verified
0             http://example.com     False     False
1  http://phishing-test-site.com      True      True
2             https://google.com     False     False
